# Теория алгоритма Ленстры–Ленстры–Ловаса (LLL)

## 1. Основные понятия о решетках

**Решетка (Lattice)** — это дискретная подгруппа в пространстве $\mathbb{R}^n$, образованная всеми целочисленными линейными комбинациями базисных векторов.

**Базис решетки** — набор линейно независимых векторов $B = \{b_1, b_2, ..., b_m\}$, через которые выражаются все векторы решетки:

$$
L(B) = \{ \sum_{i=1}^{m} x_i b_i \mid x_i \in \mathbb{Z} \}.
$$

**Проблема короткого вектора** — нахождение ненулевого вектора в решетке с минимальной длиной. Эта задача NP-трудна, но LLL дает приближенное решение.

## 2. Цель алгоритма LLL

LLL преобразует исходный базис решетки в **приведенный базис**, где векторы:

* **Почти ортогональны** (углы между ними близки к 90°).
* **Короткие** (их длины близки к минимальным в решетке).

Такие базисы полезны в:

* **Криптоанализе** (взлом RSA на основе слабых ключей).
* **Факторизации многочленов**.
* **Решении систем линейных уравнений с целыми коэффициентами**.

## 3. Условия LLL-приведенного базиса

Для параметра $\delta \in (\frac{1}{4}, 1)$ (обычно $\delta = 0.75$) базис считается LLL-приведенным, если:

**a) Size-Reduced (условие размера):**
Коэффициенты $\mu_{i,j}$, полученные при ортогонализации Грама-Шмидта, должны удовлетворять:

$$
|\mu_{i,j}| \le \frac{1}{2} \quad \text{для всех } 1 \le j < i \le m.
$$

Это гарантирует, что проекции векторов на предыдущие базисные векторы минимальны.

**b) Условие Ловаса (условие прогрессии):**
Для соседних векторов $b_k$ и $b_{k-1}$:

$$
\|b_k^*\|^2 \ge (\delta - \mu_{k,k-1}^2) \|b_{k-1}^*\|^2,
$$

где $b_i^*$ — ортогонализированные векторы, а $\mu_{k,k-1}$ — коэффициенты Грама-Шмидта.
Это условие обеспечивает "прогрессию" в длинах векторов.

## 4. Алгоритм LLL (шаги)

1.  **Ортогонализация Грама-Шмидта**
    Вычисляется ортогональный базис $\{b_1^*, b_2^*, ..., b_m^*\}$ и коэффициенты $\mu_{i,j}$:

    $$
    b_i^* = b_i - \sum_{j=1}^{i-1} \mu_{i,j} b_j^*,
    $$

    $$
    \mu_{i,j} = \frac{\langle b_i, b_j^* \rangle}{\langle b_j^*, b_j^* \rangle}.
    $$

2.  **Size Reduction (уменьшение коэффициентов)**
    Для каждого вектора $b_k$ (начиная со второго):

    Корректируем $b_k$, вычитая целые кратные предыдущих векторов:

    $$
    b_k \leftarrow b_k - \sum_{j=1}^{k-1} \lfloor \mu_{k,j} \rceil \cdot b_j,
    $$

    где $\lfloor \mu_{k,j} \rceil$ — ближайшее целое к $\mu_{k,j}$. После этой коррекции необходимо обновить коэффициенты $\mu_{k,l}$ для $l < k$.

3.  **Проверка условия Ловаса**
    Если для текущего $b_k$ условие Ловаса нарушено:

    $$
    \|b_k^*\|^2 < (\delta - \mu_{k,k-1}^2) \|b_{k-1}^*\|^2,
    $$

    меняем местами $b_k$ и $b_{k-1}$.

4.  **Итерация**
    Возвращаемся к шагу 1 (пересчитываем ортогональный базис и коэффициенты) и повторяем процесс, пока условие Ловаса не будет выполнено для всех пар соседних векторов.

## 5. Параметр $\delta$

* Чем ближе $\delta$ к 1, тем "качественнее" базис, но время работы алгоритма увеличивается.
* При $\delta = 1$ алгоритм может не завершиться.
* Стандартный выбор $\delta = 0.75$ обеспечивает баланс между скоростью и качеством.

## 6. Свойства LLL-базиса

* **Длины векторов удовлетворяют:**
    $$
    \|b_1\| \le 2^{(m-1)/2} \cdot \lambda_1,
    $$
    где $\lambda_1$ — длина кратчайшего ненулевого вектора в решетке. Первый вектор приведенного базиса гарантированно не слишком длиннее кратчайшего вектора.

* **Ортогональность:** Углы между векторами приведенного базиса относительно близки к 90°, что упрощает анализ решетки.

## 7. Пример работы

Исходный базис в $\mathbb{Z}^2$:

$$
B = \begin{pmatrix} 15 & 7 \\ 23 & 13 \end{pmatrix}.
$$

После LLL-редукции (с $\delta = 0.75$):

$$
B_{LLL} = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}.
$$


Векторы становятся ортогональными и имеют минимальную возможную длину в данной решетке (стандартная целочисленная решетка $\mathbb{Z}^2$). В более сложных решетках результат будет приведенным, но не обязательно полностью ортогональным базисом с единичными векторами.

In [ ]:
import numpy as np

def gram_schmidt(B):
    B = np.array(B, dtype=float)
    n = B.shape[0]
    m = B.shape[1]
    B_star = np.zeros((n, m))
    mu = np.zeros((n, n))
    B_norm = np.zeros(n)
    for i in range(n):
        v = B[i].copy()
        for j in range(i):
            mu[i, j] = np.dot(v, B_star[j]) / B_norm[j]
            v = v - mu[i, j] * B_star[j]
        B_star[i] = v
        B_norm[i] = np.dot(v, v)
    return B_star, mu, B_norm

def lll_reduction(B, delta=0.75):
    B = np.array(B, dtype=float)
    n = B.shape[0]
    B_star, mu, B_norm = gram_schmidt(B)
    k = 1
    while k < n:
        for j in range(k-1, -1, -1):
            q = round(mu[k, j])
            if q != 0:
                B[k] -= q * B[j]
        B_star, mu, B_norm = gram_schmidt(B)
        if B_norm[k] < (delta - mu[k, k-1]**2) * B_norm[k-1]:
            B[[k, k-1]] = B[[k-1, k]]
            k = max(k-1, 1)
            B_star, mu, B_norm = gram_schmidt(B)
        else:
            k += 1
    return B


In [36]:
basis = [[1, 1, 0], [0, 1, 1], [1, 0, 1]]
reduced_basis = lll(basis)
print("Редуцированный базис:")
print(reduced_basis)


KeyboardInterrupt: 